In [1]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import Parse, MessageToJson
import json
import psycopg2
import time
from django.contrib.gis.geos import Point, fromstr, GEOSGeometry
from datetime import datetime, timedelta
import geopandas as gpd
import rtree
import vaex
import vaex.ml
from joblib import delayed, Parallel, load, parallel_backend
from util import direction_angle, apply_dow, get_dt, get_conn, vaex_mjoin, find_trip_regex
# from haversine import haversine, Unit

dir = Path.cwd()
dir = os.path.join(dir, '../')
outdir = os.path.join(dir, 'output')
gtfs_records_zip = os.path.join(dir, "data", "GtfsRRecords.zip")
gtfs_csv_zip = os.path.join(outdir, "gtfsr_csv.zip")
gtfs_final_csv_path = os.path.join(outdir, "gtfsr.csv")
gtfs_processed_path = os.path.join(outdir, "gtfsr_processed.hdf5")
scats_model_path = os.path.join(outdir, "scats_model.json")
gtfsr_processing_temp = os.path.join(outdir, "processing_temp.hdf5")
gtfsr_arrival_means = os.path.join(outdir, "gtfsr_arrival_means.hdf5")
stop_time_data_path = os.path.join(outdir, 'stop_time_data.hdf5')
gtfs_final_hdf5_path = os.path.join(outdir, "gtfsr.csv.hdf5")

entity_cols = [
    "trip_id",
    "start_date",
    "start_time",
    "stop_sequence",
    "departure",
    "arrival",
    "timestamp",
    "stop_id",
]

In [9]:
live_df = vaex.open(os.path.join(outdir, 'deploy_gtfsr.hdf5'))
live_df

#,trip_id,start_date,start_time,stop_sequence,arrival,timestamp,stop_id,arrival_time,shape_dist_traveled,direction,route_id,lat,lon,direction_angle,shape_dist_between,arr_dow,arr_hour,arrival_mean,p_mean_vol
0,19389.1.60-155-d12-1.89.O,2.02103e+07,19:20:00,24,5,2021-03-02 19:51:26,8220DB000264,19:38:38,7818.16,0,60-155-d12-1,53.3535,-6.26226,139.315,518.6,1,19,6,68.5386


In [5]:
# vaex.open(os.path.join(outdir, 'gtfsr_model.hdf5'))

In [6]:
live_df.state_load(os.path.join(outdir, 'gtfsr_model.json'))

ModuleNotFoundError: No module named 'ml'

In [2]:
str(datetime.now())

'2021-03-04 16:43:34.367100'

In [38]:
from pandas import array

model_df = vaex.from_dict(
    {
        "trip_id": array(["19389.1.60-155-d12-1.89.O"], dtype=object),
        "start_date": array([20210302]),
        "start_time": array(["19:20:00"], dtype=object),
        "stop_sequence": array([24]),
        "arrival": array([5.0]),
        "timestamp": array(["2021-03-02 19:51:26"], dtype=object),
        "stop_id": array(["8220DB000264"], dtype=object),
        "arrival_time": array(["19:38:38"], dtype=object),
        "shape_dist_traveled": array([7818.16]),
        "direction": array(["0"], dtype=object),
        "route_id": array(["60-155-d12-1"], dtype=object),
        "lat": array([53.3535353]),
        "lon": array([-6.26225863]),
        "direction_angle": array([139.31470635]),
        "shape_dist_between": array([518.6]),
        "arr_dow": array([1]),
        "arr_hour": array([19]),
        "arrival_mean": array([6.0]),
        "p_mean_vol": array([68.53864425]),
    }
)
# model_df.state_load(gtfsr_model_path)
# model_state = model_df.state_get()

In [42]:
import io

path = os.path.join(outdir, "gtfsr_model.json")
f = open(path, 'r')

df.state_load(path)
# tmp = io.StringIO(f)
# tmp.read()
# df = vaex.open(tmp)
# df

ValueError: 'direction' is not in list